In [1]:
import pandas as pd
import json
import _load_utils
import _sql_access_utils
from fuzzywuzzy import fuzz
import re

#df = pd.read_csv('may-2021-export.csv', skiprows=8)
#print(df.size)

c:\Users\jonathan.li\Desktop\Chat Ingest\venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [18]:
lookup = _sql_access_utils.get_table_as_dataframe('dim_AL_chat_email_phone_lookup')
call_origin_new = _sql_access_utils.get_table_as_dataframe('dim_actual_lca_lookup', table_prefix = 'dbo', header=True, database = 'kcom_data')
wales = _sql_access_utils.get_table_as_dataframe('dim_actual_lca_lookup_wales', table_prefix = 'dbo', header=True, database = 'kcom_data')

In [19]:
# Get all the LCAs up until the Consortias from the Contact Details Mapping sheet.
df = _load_utils.get_dataframe_from_folder('./Contact Details Mapping', header=0, skiprows=1, sep='\t')
consortia_index = df[df['RD Name'] == 'Consortia'].iloc[0].name
df = df.drop(df.index[consortia_index-1:])
contact_details_list =  df['RD Name'].to_list()
contact_details_list = [x.replace(',','').strip() for x in contact_details_list]
dim_call_origin_new_list = call_origin_new['lca'].to_list() + wales['lca'].to_list()
lookup_list = lookup['lca'].to_list()

# LCAs in the mapping list that aren't in contact details live.
[x for x in lookup_list if x  not in contact_details_list]

# LCAs in contact details live that aren't in the mapping list.
[x for x in contact_details_list if (x not in lookup_list)]



['Citizens Advice Buckinghamshire',
 'Citizens Advice East Berkshire',
 'Citizens Advice in East Dorset and Purbeck',
 'Greater Manchester (CAGM)',
 'Guernsey Citizens Advice Bureau',
 'Jersey Citizens Advice Bureau',
 'Citizens Advice Leicestershire',
 'Citizens Advice SORT Group',
 'Surrey Welfare Rights Unit']

In [81]:
special_lcas = set([x for x in dim_call_origin_new_list if x  not in lookup_list])
special_lcas

{'Admin account',
 'AshfieldDebt',
 'Bracknell Citizens Advice',
 'Caerphilly Remote Service',
 'Cardiff Remote Service',
 'Chiltern Citizens Advice Bureau Ltd',
 'Citizens Advice Aylesbury Vale',
 'Citizens Advice Darlington',
 'Citizens Advice High Wycombe',
 'Citizens Advice Leicestershire',
 'Citizens Advice Maidenhead and Windsor',
 'Dudley District Citizens Advice Bureau',
 'East Dorset Citizens Advice',
 'Exclude: DudleyFood',
 'Flintshire (Specialist)',
 'Gwent Service',
 'Gwent service',
 'Ignore: Bhamdebt',
 'Ignore: Bhammac',
 'Liverpool Debt',
 'Manchester (National overflow)',
 'Nottinghamshire Debt',
 'Ops Team',
 'Pennine West Citizens Advice',
 'Purbeck Citizens Advice',
 'Specialist Provider Caerphilly',
 'Specialist Provider Gateshead',
 'Stockport District Citizens Advice Bureau',
 'Testing',
 'Unknown: DudleyFood'}

['Citizens Advice Walsall',
 'CLOSED',
 'lca',
 'Caerphilly National Backup Centre',
 'n/a',
 'Gateshead National Backup Centre']

In [46]:
call_origin_new

,origin,skillgroup_name,member_name,region,ops_group,member_number
0,NULL,Diss,Diss Thetford & District Citizens Advice,England,Norfolk,65/0037
1,Thetford,Diss Thetford & District,Diss Thetford & District Citizens Advice,England,Norfolk,65/0037
2,Diss,Diss Thetford & District,Diss Thetford & District Citizens Advice,England,Norfolk,65/0037
3,Doncaster,Doncaster Borough,Citizens Advice Doncaster Borough,England,Doncaster Borough,25/0044
4,Thorne,Doncaster Borough,Citizens Advice Doncaster Borough,England,Doncaster Borough,25/0044
...,...,...,...,...,...,...
599,Aldershot,Rushmoor,Rushmoor Citizens Advice Bureau,England,Single queue 25,85/0029
600,Hampshire,Rushmoor,Rushmoor Citizens Advice Bureau,England,Single queue 25,85/0029
601,Farnborough,Rushmoor,Rushmoor Citizens Advice Bureau,England,Single queue 25,85/0029
602,Ryedale,Ryedale,Citizens Advice Mid-North Yorkshire,England,Single queue 80,25/0028


In [7]:
'Leicestershire Line Chicken)'.split('(')[0].strip()

'Leicestershire Line Chicken)'